In [ ]:
# @title Install Libraries
!python -m spacy download en_trf_bertbaseuncased_lg
!python -m spacy download en_core_web_sm
!pip install spacy-transformers

In [4]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
# !cd /gdrive/My\ Drive/FYP/Datasets/DeepDatasets/csvs

Mounted at /gdrive


In [1]:
import pandas as pd
import os

In [2]:
# @title Helper Funcs

FILES_PATH =  "/gdrive/My Drive/FYP/Datasets/DeepDatasets/text/"
def takeStrength(elem):
  return elem[0][0]

def readFiles():
  filesData = {}
  FILES = os.listdir(FILES_PATH)
  for file in FILES:
    # print(file)
    with open(FILES_PATH+file, 'r') as f:
      fileData = []
      for line in f:
        sl = line.strip()
        fileData.append(str(sl))
    # print(fileData["Text"].to_list())
    filesData[file[0:len(file)-4]] = fileData
  return filesData

def readBeliefStrengths():
  bs = pd.read_csv("/gdrive/My Drive/FYP/Datasets/DeepDatasets/BELIEF_STRENGTHS.csv", header=0)
  # print(bs)
  # bs.drop(labels=["Unnamed: 4"], axis=1, inplace=True)
  return bs

# Fucntion to Change File data
def updateFileData(filePath: str, data: list):
  # data.to_csv(filePath, index=False)
  with open(filePath, 'w') as f:
    for line in data:
      f.write(line)
      f.write('\n')

def addMemoryInstance(agent: str, text: str):
  filesData = readFiles()
  data = filesData[agent]
  data.append(text)
  updateFileData("/gdrive/MyDrive/FYP/Datasets/DeepDatasets/csvs/"+agent+".csv", data)

In [5]:
# @title Set Dataset

DUMMY_DATA = {'MODUS_PONEN': ['We know that if trump uses a prevent defence strategy during the super bowl week then He will get fail. He is using the prevent defence strategy. Therefore, He will get fail.'],
                 'MODUS_TOLEN': ['If you interfere in the internal affairs of my country then I will not support you. I am not supporting you. Therefore, You have interfered in the internet affairs of my country'],
                 'INVERSE_ERROR': ['If I help you, I will not be able to vote. I did not help you. Therefore, I was able to vote.'],
                 'CONVERSE_ERROR': ['If the politician is honest, I will vote. I do manage to vote. Therefore politician was honest.']
                 }
# DATASET = DUMMY_DATA  # for dummy data
filesData = readFiles()
# filesData
DATASET = {
    'MODUS_PONEN': filesData['MODUS_PONEN'],
    'MODUS_TOLEN': filesData['MODUS_TOLEN'],
    'INVERSE_ERROR': filesData['INVERSE_ERROR'],
    'CONVERSE_ERROR': filesData['CONVERSE_ERROR']
}

In [6]:
DATASET['MODUS_PONEN'][0]

'We know that if Trump uses a prevent defence strategy during the super bowl week then He will fail. He is using the prevent defence strategy. Therefore, He will fail.'

# Actual Programs

In [7]:
import spacy 
spacy.__version__

'2.2.4'

In [8]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity as similarity

In [9]:
trf_nlp = spacy.load('en_trf_bertbaseuncased_lg')

In [25]:
#@title Initialize AGENTS_MEMORY and Default Belief Strengths
AGENTS_MEMORY = {"MODUS_PONEN": [],
                 "MODUS_TOLEN": [],
                 "INVERSE_ERROR": [],
                 "CONVERSE_ERROR": []}
AGENTS_DEF_CONF_SCORES = {"MODUS_PONEN": [1,0,0,0], "MODUS_TOLEN":[0,1,0,0], "INVERSE_ERROR":[0,0,0,1], "CONVERSE_ERROR":[0,0,0,1]}
# AGENTS_DEF_CONF_SCORES = {"MODUS_PONEN": [1], "MODUS_TOLEN":[1], "INVERSE_ERROR":[1], "CONVERSE_ERROR":[1]}

In [26]:
#@title Transform Data into Vector and Put Data and Belief Strengths Together { run: "auto", display-mode: "form" }
# BS = readBeliefStrengths()
 
for key in DATASET.keys():
  i = 0
  for example in DATASET[key]:
    score = []
    for i in AGENTS_DEF_CONF_SCORES[key]:
      score.append(i)
    AGENTS_MEMORY[key].append([trf_nlp(example).vector, score])
    i += 1

In [12]:
#@title Heart 🖤 { run: "auto", vertical-output: false, form-width: "10%" }

def classfiyLogic(inputText, inputNumber: int=0):
  SIMILARITY_THRESHOLD = 0.7

  AGENT_ID = {"MODUS_PONEN": 1, "MODUS_TOLEN": 2, "INVERSE_ERROR": 3, "CONVERSE_ERROR": 4}
  outTag = ""

  inputTextEmbeddings = trf_nlp(inputText).vector
  # print("bstrength ->", AGENTS_MEMORY["MODUS_PONEN"][len(AGENTS_MEMORY["MODUS_PONEN"])-1][1])
  # print("Last simi",similarity([inputTextEmbeddings], [AGENTS_MEMORY["MODUS_PONEN"][len(AGENTS_MEMORY["MODUS_PONEN"])-1][0]]))

  agentsWhoFoundMatches = []
  agentsContending = []
  for agent in AGENTS_DEF_CONF_SCORES.keys():
    # print(len(AGENTS_MEMORY[agent]))
    highestSim = 0.0
    matchedAgent=[]
    for i in range(len(AGENTS_MEMORY[agent])):
      sim = similarity([inputTextEmbeddings], [AGENTS_MEMORY[agent][i][0]])
      # print(sim)
      sim = sim[0][0]
      
      if (sim) > SIMILARITY_THRESHOLD:
        if sim > highestSim:
          highestSim = sim
          # print(sim)
          matchedAgent.append(agent)
          agentsContending.append(agent)
          matchedAgent.append(i)
    if len(matchedAgent) > 0:
      agentsWhoFoundMatches.append(matchedAgent)

  # print("agents are -->",agentsWhoFoundMatches)
  if len(agentsWhoFoundMatches) > 0:
    # print("contending agents are -->",agentsWhoFoundMatches)
    # Found a match
    matchedInstances = [
                          [
                          [AGENTS_MEMORY[matched[0]][matched[1]][1][(AGENT_ID[matched[0]])-1], matched[0], matched[1]],
                          AGENTS_MEMORY[matched[0]][matched[1]]
                          ]
                          for matched in agentsWhoFoundMatches
                          ]
    
    # More than one Agent participating
    if len(agentsWhoFoundMatches) > 1:
      # Sort Matched Instances based on their Belief Strength Value
      matchedInstances = sorted(matchedInstances, key= lambda elem: elem[0][0], reverse=True)

      # Multiple Agents with same belief strength
      if matchedInstances[0][0][0] == matchedInstances[1][0][0]:
        instancesWithSameStrength = []
        for i in range(len(matchedInstances)):
          if matchedInstances[i][0][0] == matchedInstances[0][0][0]:
            instancesWithSameStrength.append(matchedInstances[i])
        # User's Mudakhlat to sort the row between agents
        print("\n--------------------------------------------------------------------------------\n")
        print(f"\n The Agents have a contention on data number ->{inputNumber} \n")
        print("Multiple Agents Have matching instances from their memory, for the following text ")
        print(f"\"{inputText}\"")
        print("Please Select and Assign one class/agent from the below option: ")
        for i in range(len(instancesWithSameStrength)):
          print(f"{AGENT_ID[instancesWithSameStrength[i][0][1]]} = {instancesWithSameStrength[i][0][1]}")
        try:
          userAssignedVal = int(input("Your Answer Here : --> "))
          userAgentName = [k for k, v in AGENT_ID.items() if v == userAssignedVal]
          print('User Input ',userAgentName[0])
          print("\n--------------------------------------------------------------------------------\n")
          # Set Agents' memories according to user input
          finalInstanceAfterUserInput = [instance for instance in instancesWithSameStrength if instance[0][1] == userAgentName[0]]
          # AGENTS_MEMORY[finalInstanceAfterUserInput[0][0][1]][finalInstanceAfterUserInput[0][0][2]][1][(AGENT_ID[finalInstanceAfterUserInput[0][0][1]])-1] += 1
          
          # Set instances' beliefs
          for i in range(0, len(matchedInstances)):
            AGENTS_MEMORY[matchedInstances[i][0][1]][matchedInstances[i][0][2]][1][(AGENT_ID[userAgentName[0]])-1] += 1
          
          # Now set other instances' belief strength (other than winning agent)
          outTag = finalInstanceAfterUserInput[0][0][1]
        except Exception as e:
          print("\n--------------------------------------------------------------------------------\n")
          print("💀💀  Please Enter a number (integer)  💀💀\n", e)
          print("\n--------------------------------------------------------------------------------\n")
      else:
        outTag = matchedInstances[0][0][1]
        AGENTS_MEMORY[matchedInstances[0][0][1]][matchedInstances[0][0][2]][1][(AGENT_ID[matchedInstances[0][0][1]])-1] += 1

    # only one agent is participating
    else:
      outTag = matchedInstances[0][0][1]
      AGENTS_MEMORY[matchedInstances[0][0][1]][matchedInstances[0][0][2]][1][(AGENT_ID[matchedInstances[0][0][1]])-1] += 1
  
  # Now deal with the competition between agents
  else:
    print("\n--------------------------------------------------------------------------------\n")
    print(f"\n The Agents have no match on data number ->{inputNumber} \n")
    # could not find ant match -> User's Mudaakhlat
    print("No agent was able to classify, please specify a class (out of following four agents) for the Following Text")
    print(f"\"{inputText}\"")
    print("1 = Modus Ponen \t 2 = Modus Tolen \n3 = Inverse Error \t 4 = Converse Error")
    try:
      userChoice = int(input("Enter One of the Above Options --> "))
      print("\n--------------------------------------------------------------------------------\n")
      userAgentName = [k for k, v in AGENT_ID.items() if v == userChoice]
      outTag = userAgentName[0]
      score = []
      for i in AGENTS_DEF_CONF_SCORES[userAgentName[0]]:
        score.append(i)
      AGENTS_MEMORY[userAgentName[0]].append([inputTextEmbeddings, [1565]])
    except Exception as e:
        print("\n--------------------------------------------------------------------------------\n")
        print("💀💀  Please Enter a number (integer)  💀💀", str(e))
        print("\n--------------------------------------------------------------------------------\n")

  # print("Final Output ", outTag)
  return outTag

In [33]:

4# @title Run Demo on Single Tweet/Text

inputText = str(input("Enter a text to classfiy: "))
classOfText = classfiyLogic(inputText)
print(f"Output Logical Class of the sentence is --> {classOfText}")

Enter a text to classfiy: If I do not vote for you, I will be helping Biden. I have helped Biden. Therefore, I have not voted for you
Output Logical Class of the sentence is --> CONVERSE_ERROR


In [ ]:
# AGENTS_MEMORY["MODUS_PONEN"]

# Testing And Experimenatation

In [ ]:
#@title Read Test File { form-width: "10%" }
import pandas as pd
df = pd.read_csv("/gdrive/MyDrive/FYP/Datasets/logic_test_data.csv", names=["Text"])
test_data = df["Text"].to_list()
test_data

In [19]:
#@title Test 100 Instances (for Learning Trend)
for i in range(len(test_data)):
  classOfText = classfiyLogic(test_data[i], i)
  print("______________________RESULT__________________________")
  print(f"Logical Class of Text Instance {i} is --> {classOfText} ")
  print("______________________END RESULT_______________________")



--------------------------------------------------------------------------------


 The Agents have a contention on data number ->0 

Multiple Agents Have matching instances from their memory, for the following text 
"It’s Super Bowl week, and Trump is using a prevent defense. The strategy sometimes backfires iN football. It may fail for Trump, too."
Please Select and Assign one class/agent from the below option: 
1 = MODUS_PONEN
2 = MODUS_TOLEN
4 = CONVERSE_ERROR
Your Answer Here : --> 1
User Input  MODUS_PONEN

--------------------------------------------------------------------------------

______________________RESULT__________________________
Logical Class of Text Instance 0 is --> MODUS_PONEN 
______________________END RESULT_______________________
______________________RESULT__________________________
Logical Class of Text Instance 1 is --> MODUS_PONEN 
______________________END RESULT_______________________

---------------------------------------------------------------------

KeyboardInterrupt: ignored

In [ ]:
for agent in AGENTS_DEF_CONF_SCORES.keys():
  for memInstance in AGENTS_MEMORY[agent]:
    print(memInstance[1])